In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)

2.6.0


In [5]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [6]:
nb_classes = 3
print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [10]:
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 2.0265348 , -0.19990598,  0.187595  ],
       [-1.8624718 ,  1.1830902 , -0.75108314],
       [ 0.7819291 ,  0.19707595,  0.6640797 ],
       [ 1.5643852 , -0.04990807, -0.38255563]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.4564867 ,  0.53983474, -1.1366715 ], dtype=float32)>


In [11]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[2.4881816e-02 9.6873927e-01 6.3789249e-03]
 [9.8632777e-01 1.2180602e-02 1.4916508e-03]
 [9.9994230e-01 5.2787913e-05 4.8965267e-06]
 [9.9999952e-01 3.5200765e-07 8.7773429e-08]
 [4.1362935e-05 9.9995792e-01 7.1108070e-07]
 [9.9881989e-01 1.1706438e-03 9.4601755e-06]
 [7.7800588e-03 9.9221438e-01 5.5833375e-06]
 [3.3521624e-03 9.9664682e-01 9.2721189e-07]], shape=(8, 3), dtype=float32)


In [13]:
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[1.0000000e+00 5.2308369e-08 1.9130997e-09]], shape=(1, 3), dtype=float32)


In [15]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.994226, shape=(), dtype=float32)


In [19]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [22]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 1.0009193 , -0.2520938 , -0.7488258 ],
       [-0.98698664,  1.4851966 , -0.4982102 ],
       [ 0.3810175 ,  0.49261224, -0.87363005],
       [ 0.5075718 ,  0.36624253, -0.8738147 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.2526431 ,  0.12137035, -0.37401348], dtype=float32)>]


In [23]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 6.239289
Loss at epoch 100: 0.677962
Loss at epoch 200: 0.582576
Loss at epoch 300: 0.524000
Loss at epoch 400: 0.477279
Loss at epoch 500: 0.436237
Loss at epoch 600: 0.398209
Loss at epoch 700: 0.361562
Loss at epoch 800: 0.325099
Loss at epoch 900: 0.288242
Loss at epoch 1000: 0.255323
Loss at epoch 1100: 0.239535
Loss at epoch 1200: 0.227773
Loss at epoch 1300: 0.217080
Loss at epoch 1400: 0.207312
Loss at epoch 1500: 0.198354
Loss at epoch 1600: 0.190108
Loss at epoch 1700: 0.182493
Loss at epoch 1800: 0.175442
Loss at epoch 1900: 0.168893
Loss at epoch 2000: 0.162797


In [25]:
sample_data = [[2,1,3,2]] 
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) 

tf.Tensor([[0.00127107 0.08592235 0.91280663]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [26]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[1.81193195e-06 1.06663001e-03 9.98931587e-01]
 [1.27106626e-03 8.59223828e-02 9.12806511e-01]
 [3.72801821e-07 1.69231981e-01 8.30767632e-01]
 [1.18178705e-05 8.44869196e-01 1.55119017e-01]
 [2.63901263e-01 7.23615468e-01 1.24832708e-02]
 [1.38862044e-01 8.61132622e-01 5.30040597e-06]
 [7.39666760e-01 2.60313153e-01 2.00841187e-05]
 [9.21692789e-01 7.83068687e-02 2.93730267e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [27]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 3.223562
Loss at epoch 500: 0.451360
Loss at epoch 1000: 0.236338
Loss at epoch 1500: 0.186408
Loss at epoch 2000: 0.153952
